In [1]:
import pymysql
import numpy as np
import pandas as pd
import pandas as pd
import os, json, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()

In [2]:
filename = 'https://datasets.imdbws.com/title.basics.tsv.gz'
df = pd.read_csv(filename, sep='\t', low_memory=False)

df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [6]:
# replacing \N
df = df.replace({'\\N':np.nan})

In [7]:
df= df.dropna(subset = ['runtimeMinutes','genres'])

In [8]:
df = df.loc[(df['startYear'] >= '2000')]

In [9]:
df['startYear'] = df['startYear'].astype(float)

In [14]:
# df.dropna(inplace = True)
# df.head(50)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
108563,tt0111056,tvSeries,Gensomaden Saiyuki,Gensomaden Saiyuki,0,2000,2001,23,"Action,Adventure,Animation"
137836,tt0142032,tvMiniSeries,Dune,Dune,0,2000,2000,265,"Adventure,Drama,Sci-Fi"
153556,tt0158466,tvMiniSeries,Anne of Green Gables: The Continuing Story,Anne of Green Gables: The Continuing Story,0,2000,2000,185,"Drama,Family,Romance"
155749,tt0160904,tvSeries,MI-5,Spooks,0,2002,2011,60,"Action,Crime,Drama"
158616,tt0163944,tvMiniSeries,Labyrinten,Labyrinten,0,2000,2000,259,"Drama,Thriller"
162047,tt0167701,tvSeries,Randall & Hopkirk (Deceased),Randall & Hopkirk (Deceased),0,2000,2001,50,"Comedy,Crime,Fantasy"
171933,tt0178140,tvMiniSeries,The Wide Net,Det grovmaskiga nätet,0,2000,2000,176,"Crime,Drama,Mystery"
171953,tt0178161,tvSeries,The Secret Adventures of Jules Verne,The Secret Adventures of Jules Verne,0,2000,2000,60,"Adventure,Horror,Sci-Fi"
172796,tt0179051,tvMiniSeries,Ted and Alice,Ted and Alice,0,2002,2002,180,"Comedy,Drama,Romance"
174784,tt0181199,tvMiniSeries,Arabian Nights,Arabian Nights,0,2000,2000,175,"Adventure,Family,Fantasy"


In [12]:
df.titleType.value_counts()

tvEpisode       879985
short           479525
movie           221311
video           149475
tvSeries         61977
tvMovie          47585
tvMiniSeries     12338
tvSpecial        12329
tvShort           5712
videoGame          271
Name: titleType, dtype: int64

In [10]:
df.loc[(df['titleType'] == 'movie')]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021.0,NaN,133,Documentary
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
66336,tt0067683,movie,Workers '71: Nothing About Us Without Us,Robotnicy 1971 - Nic o nas bez nas,0,2006.0,NaN,47,Documentary
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
...,...,...,...,...,...,...,...,...,...
9492579,tt9916362,movie,Coven,Akelarre,0,2020.0,NaN,92,"Drama,History"
9492663,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019.0,NaN,123,Drama
9492704,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015.0,NaN,57,Documentary
9492731,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007.0,NaN,100,Documentary


In [13]:
is_documentary = df['genres'].str.contains('documentary',case=False)
df = df[~is_documentary]

In [14]:
file_2 = 'https://datasets.imdbws.com/title.akas.tsv.gz'
akas = pd.read_csv(file_2, sep='\t', low_memory=False)

In [ ]:
akas = akas.replace({'\\N':np.nan})

In [ ]:
akas.loc[(akas['region'] == 'US')]

In [ ]:
keepers =df['tconst'].isin(akas['titleId'])
keepers

In [ ]:
df = df[keepers]
df

In [ ]:
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

In [ ]:
df.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [ ]:
df = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
df.head()

RATING

In [15]:
file_3 = 'https://datasets.imdbws.com/title.ratings.tsv.gz'
Ratings = pd.read_csv(file_3, sep='\t', low_memory=False)

In [16]:
Ratings = Ratings.replace({'\\N':np.nan})

In [17]:
Ratings.head(5)

,tconst,averageRating,numVotes
0,tt0000001,5.7,1930
1,tt0000002,5.8,261
2,tt0000003,6.5,1750
3,tt0000004,5.6,176
4,tt0000005,6.2,2562


In [18]:
keeper =Ratings['tconst'].isin(akas['titleId'])
keeper

0           True
1           True
2           True
3           True
4           True
           ...  
1262150    False
1262151    False
1262152     True
1262153     True
1262154    False
Name: tconst, Length: 1262155, dtype: bool

In [19]:
Ratings = Ratings[keeper]
Ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1930
1,tt0000002,5.8,261
2,tt0000003,6.5,1750
3,tt0000004,5.6,176
4,tt0000005,6.2,2562
...,...,...,...
1262142,tt9916460,9.4,18
1262144,tt9916538,8.3,6
1262145,tt9916544,6.9,61
1262152,tt9916730,8.0,8
